In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from collections import Counter

In [58]:
data = pd.read_csv("train.csv")

In [59]:
data1 = data[["bathrooms", "bedrooms", "latitude", "longitude", "price"]]
data1.loc[:, "bathrooms"] = data1.loc[:, "bathrooms"].astype("int")
# количество фото
data1["photos"] = data.photos.str.len()

C:\Users\poisk\AppData\Local\Temp\ipykernel_11816\3198113518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.loc[:, "bathrooms"] = data1.loc[:, "bathrooms"].astype("int")
C:\Users\poisk\AppData\Local\Temp\ipykernel_11816\3198113518.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["photos"] = data.photos.str.len()


In [65]:
data.TARGET.value_counts(normalize=True)


low       0.694697
medium    0.227523
high      0.077780
Name: TARGET, dtype: float64

In [60]:
X1 = data1
y = data.TARGET

Датасет несбалансированный, поэтому установим class_weight равным balanced

In [61]:
logreg = LogisticRegression(class_weight="balanced", max_iter=1000)
np.mean(cross_val_score(logreg, X1, y, cv=5, scoring="balanced_accuracy"))

0.4937424763008133

Дата создания:

In [11]:
data.created = pd.to_datetime(data.created)
X = pd.DataFrame(data1, copy=True)
X["day"] = data.created.dt.day_of_week
X = X.drop(columns="day")
X["month"] = data.created.dt.month
X = X.drop(columns="month")

Количество комнат:

In [12]:
X["rooms"] = X.bathrooms + X.bedrooms

In [13]:
logreg = LogisticRegression(class_weight="balanced")
np.mean(cross_val_score(logreg, X, y, cv=5, scoring="balanced_accuracy"))

c:\users\poisk\pycharmprojects\machine_learning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\poisk\pycharmprojects\machine_learning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

0.4947220265505067

Выберем 50 самых часто встречающихся текстовых признаков и добавим

In [17]:
data["feats_lst"] = data.features.apply(lambda x: x[1: -2])
data.feats = data.feats.str.split(", ")
features = []

for i in data.feats:
    features += i

total = sorted(Counter(features).items(), key=lambda x: x[1])

lst = [i[0] for i in total[-51:] if i[0]]

for i in lst:
    X[i] = 0
    X.loc[data.features.str.contains(i), i] = 1

Добавим расстояние от центра (примерного)

In [19]:
X["rooms"] = X.bathrooms + X.bedrooms

center_lat = 40.785
center_long = -73.9683
X["center_dist"] = ((center_lat - X.latitude) ** 2 + (center_long - X.longitude) ** 2) ** 0.5

In [20]:
logreg = LogisticRegression(class_weight="balanced")
np.mean(cross_val_score(logreg, X, y, cv=5, scoring="balanced_accuracy"))

c:\users\poisk\pycharmprojects\machine_learning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\poisk\pycharmprojects\machine_learning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

0.5143916851143434

In [24]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [26]:
logreg = LogisticRegression(class_weight="balanced")
np.mean(cross_val_score(logreg, X_scaled, y, cv=5, scoring="balanced_accuracy"))

c:\users\poisk\pycharmprojects\machine_learning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\poisk\pycharmprojects\machine_learning\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

0.5382875793581183

In [54]:
# подберем параметр для логистической регрессии
regulars = np.arange(7, 10, 0.5)
for regular in regulars:
    logreg = LogisticRegression(class_weight="balanced", C=regular, max_iter=1000)
    print(regular, np.mean(cross_val_score(logreg, X_scaled, y, cv=3, scoring="balanced_accuracy")))

7.0 0.541648537349794
7.5 0.542047459962702
8.0 0.5425387950943014
8.5 0.542400761448879
9.0 0.5425272457576152
9.5 0.5427360730725209


Лучшее C = 9.5

In [48]:
test = pd.read_csv("test.csv")

In [49]:
test.head()

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,low,2.0,3,87c4e08083ac83618dd9fdbf849331fe,2016-06-21 03:21:59,Rent listed is net effective after one month f...,E 77 St.,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7716,7189305,-73.9544,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7189305_f47ed5c...,4675,339 E 77 St.
1,low,1.0,2,850c3d4412aebb3335273ed95f3e4bd4,2016-06-11 03:47:06,A FABULOUS 2BR IN THE UPPER EAST SIDE! PERFECT...,E 112 Street,[],40.7944,7140670,-73.9401,3e9f0fa34d67e5d61aae56776ecb8cee,['https://photos.renthop.com/2/7140670_6c5824f...,2100,252 E 112 Street
2,low,1.0,2,84ddf917a091828ab9baedc0c926470e,2016-04-17 02:45:19,This is a great true 2 bedroom in a prime Midt...,Ninth Avenue,"['Dogs Allowed', 'Cats Allowed']",40.7650,6886239,-73.9882,10b1ae0a38d50b7ba0cee612d14af9eb,['https://photos.renthop.com/2/6886239_533bbee...,2800,783 Ninth Avenue
3,high,2.0,3,315a2f868a195b076e5be73411110da2,2016-05-07 11:44:35,Brand new custom renovation in beautiful three...,East 12th street,"['Laundry In Unit', 'No Fee', 'Washer/Dryer in...",40.7288,6981146,-73.9808,e9920062e07ee893c10e38d0259665b0,['https://photos.renthop.com/2/6981146_f4a7400...,5500,186 Avenue B
4,medium,1.0,3,9b525bacc8d294728c35f27c04e22f46,2016-06-24 07:33:26,"Welcome Home! Located on a Prime UWS Block, Ba...",West 86th&Columbus,"['Private Outdoor Space', 'Elevator', 'Laundry...",40.7864,7210864,-73.9713,6d389fbe372d4d30b17733caa9370f95,['https://photos.renthop.com/2/7210864_e8528f4...,3995,47 West 86th street


In [51]:
test.photos = test.photos.apply(lambda x: x[1: -2])
test.photos = test.photos.str.split(", ")
X_test = test[["bathrooms", "bedrooms", "latitude", "longitude", "price"]]
X_test.loc[:, "bathrooms"] = X_test.loc[:, "bathrooms"].astype("int")
X_test["photos"] = test.photos.str.len()
X_test["rooms"] = X_test.bathrooms + X_test.bedrooms
X_test["center_dist"] = ((center_lat - X_test.latitude) ** 2 + (
        center_long - X_test.longitude) ** 2) ** 0.5

for i in lst:
    X_test[i] = 0
    X_test.loc[test.features.str.contains(i), i] = 1


C:\Users\poisk\AppData\Local\Temp\ipykernel_11816\29518565.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:, "bathrooms"] = X_test.loc[:, "bathrooms"].astype("int")
C:\Users\poisk\AppData\Local\Temp\ipykernel_11816\29518565.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["photos"] = test_df.photos.str.len()
C:\Users\poisk\AppData\Local\Temp\ipykernel_11816\29518565.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [52]:
logreg = LogisticRegression(class_weight="balanced", C=9.5, max_iter=3000).fit(X_scaled, y)
prediction = logreg.predict(X_test_scaled)

In [63]:
ids = pd.read_csv("sample_submission.csv")
pred_df = pd.DataFrame({"Id": ids.Id, "TARGET": prediction}).to_csv("prediction.csv", index=False)

Score: 0.53842
Public score: 0.55041